In [1]:
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import sys
sys.path.append('../src')
from sharedcontrolpaper.force_sensitive_stopping_task_utils import get_subject_label, string_to_numbers, process_trial_data

In [2]:
parent_directory = os.path.dirname(os.getcwd())
data_path = os.path.join(parent_directory, 'data', 'experiment')
task = "force_sensitive_stopping_task"
exp_stage = "final"
pattern = os.path.join(data_path, exp_stage, '*', task, '*.csv')
data_files = glob.glob(pattern)

In [3]:
shared_control_metrics = {}

for file in data_files:
    subject_label = get_subject_label(file)
    
    df = pd.read_csv(file)
    
    # some post processing
    df['block'] = df['block'].str.strip("'")
    # create a dataFrame for each block
    df_test = df.query("block != 'practice'")
    block_1 = df_test.query("block == 'block 1'")
    block_2 = df_test.query("block == 'block 2'")

    block_1 = block_1.reset_index(drop=True)
    block_2 = block_2.reset_index(drop=True)

    task_dfs = [block_1, block_2]
    
    for df in task_dfs:
        if 'AI-engaged' in df['condition'].values:
            ai_data = df.copy()
            ai_data['distances'] = ai_data['distances'].apply(string_to_numbers)
            ai_data['pressures'] = ai_data['pressures'].apply(string_to_numbers)
            ai_data['time_stamps'] = ai_data['time_stamps'].apply(string_to_numbers)
        else:
            non_ai_data = df.copy()
            non_ai_data['distances'] = non_ai_data['distances'].apply(string_to_numbers)
            non_ai_data['pressures'] = non_ai_data['pressures'].apply(string_to_numbers)
            non_ai_data['time_stamps'] = non_ai_data['time_stamps'].apply(string_to_numbers)
    
    shared_control_metrics[subject_label] = {'AI': {'data': ai_data}, 'Non-AI': {'data': non_ai_data}}
    
    for block in shared_control_metrics[subject_label].keys():
        trial_results, ssrt_list = process_trial_data(shared_control_metrics[subject_label][block]['data'], block=block)
        shared_control_metrics[subject_label][block]['trial_results'] = trial_results
        shared_control_metrics[subject_label][block]['ssrt_list'] = ssrt_list 


In [4]:
%store shared_control_metrics

Stored 'shared_control_metrics' (dict)


## Excluded Observations

In [7]:
exclusions = {"s027": ["AI", 80, 96]}

## Grabbing SSRT and other metrics across conditions

In [19]:
def grab_mean_metric(measure):

    """
    Function to find the mean of a specified metric across different trial conditions 
    for each subject. The provided measure should be a string representing the metric 
    to analyze (e.g., 'ssrt' or 'duration_of_inhibition'). The results are saved to a 
    CSV file with each row corresponding to a subject, with columns for the means 
    of the measure in each of the three trial conditions.

    Parameters:
    - measure (str): The name of the measure to calculate the mean for.

    Outputs:
    - Saves a CSV file with means for each subject across non_ai trials, ai_condition_stop_trials, 
      and ai_condition_ai_trials, as well as handling specific conditions based on the flag.
    """

    condition_measure = {}

    for subject in shared_control_metrics.keys():
        non_ai_trials = []
        ai_disengaged_trials = []
        ai_engaged_trials = []
        
        for block in shared_control_metrics[subject].keys():
            
            for trial in shared_control_metrics[subject][block]['trial_results'].keys():

                if subject in exclusions.keys() and trial in exclusions[subject] and block in exclusions[subject]:
                    continue
        

                if block == 'Non-AI':
                    non_ai_trials.append(shared_control_metrics[subject][block]['trial_results'][trial][measure])
                    
                elif (block == 'AI') and (shared_control_metrics[subject][block]['trial_results'][trial]['condition'] == 'AI-disengaged'):
                    ai_disengaged_trials.append(shared_control_metrics[subject][block]['trial_results'][trial][measure])
                    
                elif (block == 'AI') and (shared_control_metrics[subject][block]['trial_results'][trial]['condition'] == 'AI-engaged'):
                    ai_engaged_trials.append(shared_control_metrics[subject][block]['trial_results'][trial][measure])
                    

        avg_ai_engaged = np.nanmean(ai_engaged_trials)
        avg_ai_disengaged = np.nanmean(ai_disengaged_trials)
        avg_non_ai = np.nanmean(non_ai_trials)  
        
        condition_measure[subject] = {f'non_ai': avg_non_ai, f'ai_disengaged': avg_ai_disengaged, f'ai_engaged': avg_ai_engaged}

    df = pd.DataFrame(condition_measure).T
    df = df.sort_index()
    return df

In [20]:
force_sensitive_stopping_task_ssrt = grab_mean_metric('ssrt')
%store force_sensitive_stopping_task_ssrt

Stored 'force_sensitive_stopping_task_ssrt' (DataFrame)


In [8]:
duration_of_inhibition = grab_mean_metric('duration_of_inhibition')
%store duration_of_inhibition

Stored 'duration_of_inhibition' (DataFrame)


In [9]:
go_task_accuracy_before_stop_onset = grab_mean_metric('go_task_accuracy_before_stop_onset')
%store go_task_accuracy_before_stop_onset

Stored 'go_task_accuracy_before_stop_onset' (DataFrame)


In [10]:
go_task_accuracy_after_stop_onset = grab_mean_metric('go_task_accuracy_after_stop_onset')
%store go_task_accuracy_after_stop_onset

Stored 'go_task_accuracy_after_stop_onset' (DataFrame)


In [11]:
ball_before_ring_proportion_before_stop_onset = grab_mean_metric('ball_before_ring_proportion_before_stop_onset')
print(ball_before_ring_proportion_before_stop_onset.tail(1))

        non_ai  ai_disengaged  ai_engaged
s043  0.209611        0.32973    0.173612


In [12]:
ball_after_ring_proportion_before_stop_onset = grab_mean_metric('ball_after_ring_proportion_before_stop_onset')
print(ball_after_ring_proportion_before_stop_onset.tail(1))

      non_ai  ai_disengaged  ai_engaged
s043     0.0            0.0         0.0


## Finding the proportion of trials where subjects inhibited

In [13]:
def find_proportion_metric(metric):
    proportion = {}

    for subject in shared_control_metrics.keys():
        count_non_ai = 0
        count_ai_disengaged = 0
        count_ai_engaged = 0
        for block in shared_control_metrics[subject].keys():
            
            for trial in shared_control_metrics[subject][block]['trial_results'].keys():

                if subject in exclusions.keys() and trial in exclusions[subject] and block in exclusions[subject]:
                    continue

                if (block == 'Non-AI'):
                    if not np.isnan(shared_control_metrics[subject][block]['trial_results'][trial][metric]):
                        count_non_ai += 1
                
                elif (block == 'AI') and (shared_control_metrics[subject][block]['trial_results'][trial]['condition'] == 'AI-disengaged'):
                    if not np.isnan(shared_control_metrics[subject][block]['trial_results'][trial][metric]):
                        count_ai_disengaged += 1
                        
                elif (block == 'AI') and (shared_control_metrics[subject][block]['trial_results'][trial]['condition'] == 'AI-engaged'):
                    if not np.isnan(shared_control_metrics[subject][block]['trial_results'][trial][metric]):
                        count_ai_engaged += 1

        proportions_non_ai = count_non_ai / 100
        proportions_ai_disengaged = count_ai_disengaged / 20
        proportions_ai_engaged = count_ai_engaged / 80
        
        proportion[subject] = {
            'proportion_non_ai': proportions_non_ai,
            'proportion_ai_disengaged': proportions_ai_disengaged,
            'proportion_ai_engaged': proportions_ai_engaged
        }

    df = pd.DataFrame(proportion).T
    df = df.sort_index()
    df.loc['mean'] = df.mean()
    print(df.loc['mean'])



In [14]:
find_proportion_metric('ssrt')

proportion_non_ai           0.999750
proportion_ai_disengaged    0.996250
proportion_ai_engaged       0.989062
Name: mean, dtype: float64


## Create CSVs of SSRT by each half of trials in a block 

In [15]:
def grab_mean_metric_by_halves(measure):
    """
    Function to find the mean of a specified metric across different trial conditions 
    for each subject, split by halves of trials.

    Parameters:
    - measure (str): The name of the measure to calculate the mean for.

    Outputs:
    - Saves two CSV files with means for each subject across non_ai trials and ai condition trials
      split by halves.
    """
    # Initialize structures for data collection
    condition_measure_first_half = {}
    condition_measure_second_half = {}

    for subject in shared_control_metrics.keys():
        non_ai_first_half = []
        non_ai_second_half = []
        ai_disengaged_first_half = []
        ai_disengaged_second_half = []
        ai_engaged_first_half = []
        ai_engaged_second_half = []
        
        for block in shared_control_metrics[subject].keys():
            trial_results = shared_control_metrics[subject][block]['trial_results']
            num_trials = len(trial_results)

            for index, trial in enumerate(trial_results.keys()):
                if subject in exclusions.keys() and trial in exclusions[subject] and block in exclusions[subject]:
                    continue
                
                ssrt_value = shared_control_metrics[subject][block]['trial_results'][trial][measure]
                
                if pd.isna(ssrt_value):
                    continue
                
                if block == 'Non-AI':
                    if index < num_trials / 2:  # First half
                        non_ai_first_half.append(ssrt_value)
                    else:  # Second half
                        non_ai_second_half.append(ssrt_value)
                    
                elif block == 'AI':
                    condition = shared_control_metrics[subject][block]['trial_results'][trial]['condition']
                    if condition == 'AI-disengaged':
                        if index < num_trials / 2:  # First half
                            ai_disengaged_first_half.append(ssrt_value)
                        else:  # Second half
                            ai_disengaged_second_half.append(ssrt_value)
                    elif condition == 'AI-engaged':
                        if index < num_trials / 2:  # First half
                            ai_engaged_first_half.append(ssrt_value)
                        else:  # Second half
                            ai_engaged_second_half.append(ssrt_value)
                            
        condition_measure_first_half[subject] = {
            'non_ai': np.nanmean(non_ai_first_half),
            'ai_disengaged': np.nanmean(ai_disengaged_first_half),
            'ai_engaged': np.nanmean(ai_engaged_first_half)
        }
        
        condition_measure_second_half[subject] = {
            'non_ai': np.nanmean(non_ai_second_half),
            'ai_disengaged': np.nanmean(ai_disengaged_second_half),
            'ai_engaged': np.nanmean(ai_engaged_second_half)
        }
    
    df_first_half = pd.DataFrame(condition_measure_first_half).T
    df_second_half = pd.DataFrame(condition_measure_second_half).T

    df_first_half = df_first_half.sort_index()

    df_second_half = df_second_half.sort_index()
    
    return df_first_half, df_second_half

In [16]:
ssrt_first_half, ssrt_second_half = grab_mean_metric_by_halves('ssrt')
%store ssrt_first_half
%store ssrt_second_half

Stored 'ssrt_first_half' (DataFrame)
Stored 'ssrt_second_half' (DataFrame)
